# SQLite3 as a data loader

Before everything else, you should go to SQLite3's [official page](https://www.sqlite.org/cli.html) and review the built-in commands - they will be, if not currently are, your great efficiency tool.

Make sure you understand at least the following:

- `.schema` and `.tables` for listing the database's metadata
- `.mode` for altering output mode
- `.header` for toggling header
- `.output` for redirecting output (esp. the result from `SELECT`) to a file

## loading csv file

SQLite has a built-in function to load CSV data into a table.
```sh
sqlite3 my_database.db <<EOF
.mode csv
.import "./data/airports.csv"
EOF
```
note the usage of `EOF` (i.e. [heredoc](https://en.wikipedia.org/wiki/Here_document)) as a handy way to pipe multiple lines of input into a process. The shell command above starts a `sqlite3` process which saves data to `my_database.db` and takes two commands.